# PISA Data Exploration
## by Simon Hwang

## Preliminary Wrangling
The Programme for International Student Assessment (PISA) is a means of measuring 15-year-old student ability in reading, mathematics, and science. The assessments are conducted every 3 years since 2000, and usually garner participation from about 80 countries.

In [ ]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

In [ ]:
pisa = pd.read_csv('pisa2012.csv',sep=',', encoding='latin-1',error_bad_lines=False, index_col=False, dtype='str')

In [ ]:
pisa.head()

In [ ]:
pisa.info()

In [ ]:
dic = pd.read_csv('pisadict2012.csv', encoding='latin-1')

In [ ]:
dic

### Narrow the dataset into a subset that is simplified and easier to work with, using only OECD countries and key columns to my investigation

In [ ]:
df = pisa.drop(pisa[pisa.OECD == 'Non-OECD'].index).copy()

In [ ]:
df.OECD.isin(['OECD']).all()

In [ ]:
df = df[['CNT','ST04Q01','ESCS','PV1MATH','PV2MATH','PV3MATH','PV4MATH','PV5MATH','PV1READ','PV2READ','PV3READ','PV4READ','PV5READ','PV1SCIE','PV2SCIE','PV3SCIE','PV4SCIE','PV5SCIE']]

In [ ]:
df.head()

In [ ]:
# check the countries that are left, and their sample sizes
df.CNT.value_counts()

In [ ]:
# verify data completeness and dtypes
df.info()

In [ ]:
# investigate missing ESCS values
df[df.ESCS.isna()].sample(5)

In [ ]:
df[df.ESCS.isna()].CNT.value_counts()

In [ ]:
# drop nan rows, as it is a problem that seems to affect most OECD countries to some degree
df.dropna(inplace=True)

In [ ]:
# update data types that are numbers
for i in range(2, 18):
    df.iloc[:, i] = df.iloc[:, i].astype('float64')

In [ ]:
# check dtypes are corrected
df.info()

In [ ]:
# computer PISA scores by averaging plausible values. See readme for flaws of this approach
df['MathScore'] = df.loc[:,'PV1MATH':'PV5MATH'].mean(axis=1)
df['ReadScore'] = df.loc[:,'PV1READ':'PV5READ'].mean(axis=1)
df['ScienceScore'] = df.loc[:,'PV1SCIE':'PV5SCIE'].mean(axis=1)

In [ ]:
# drop plausible value columns
df = df.iloc[:,[0, 1, 2, 18, 19, 20]]

In [ ]:
df.columns = ['Country', 'Sex', 'SocioEconIndex', 'MathScore', 'ReadScore', 'ScienceScore']

In [ ]:
df.head()

### What is the structure of your dataset?

This dataset was imported as a flat file with 636 columns and 485490 rows. I have converted it into a clean, tidy subset of that file.

### What is/are the main feature(s) of interest in your dataset?
There are many possible main features of interest in this dataset, but I will focus on the scores in mathematics, reading, and science given to each student. PISA gives 5 plausible values for each knowledge area which I will average. Ordinarily, these should be weighted and replicated as per the PISA Data Analysis Manual. However, this is beyond the scope of this project, so I will be using them as is.
> Note: any relationships and conclusions may be invalid due to the discrepancy in error resulting from failing to adhere to the data wrangling steps detailed in their guidelines. See the `readme` for more information.

### What features in the dataset do you think will help support your investigation into your feature(s) of interest?
The name of the country (CNT), gender of the student (ST04Q01), and the student's index of economic, social and cultural status (ESCS).

## Univariate Exploration

In [ ]:
df.Country.describe()

In [ ]:
plt.figure(figsize=(16,8))
plt.barh(width=df.Country.value_counts(), y=df.Country.value_counts().index);

In [ ]:
plt.figure(figsize=(10, 10))
plt.pie(df.Country.value_counts(), labels=df.Country.value_counts().index);

Student responses span a wide range, but do not fall far below 5000, probably because most OECD countries opt to obey PISA's __5000 students minimum requirement__. Iceland is somewhat of an outlier at the low end, but is excused due to its low population, which makes it difficult to reach 5000 even if they test every 15 year old student in the whole country. Canada, Spain, Italy, and Mexico all test well above 20000. This likely will make a good graph, with some polishing, to show off in the final report.

I also graphed a pie chart, which I think helps show how the higher sample sizes of Mexico, Italy, and Spain account for more than a quarter of the total sample. This would cause significant error, since I will not be using the PISA student weights.

In [ ]:
plt.pie(df.Sex.value_counts(), labels=df.Sex.value_counts().index, startangle=90, autopct='%1.2f%%');

The sex of students in this sample is decidedly female-biased, but it's important to note that this may be biased by those countries that test more students. This could be an interesting investigation for bivariate exploration. Also, with such an even distribution, this will make for a great categorical variable to layer on top of bivariate exploration to see multivariate relationships.

In [ ]:
df.SocioEconIndex.describe()

In [ ]:
bins = np.arange(-4, 3.5, 0.5)
plt.hist(data=df, x='SocioEconIndex', bins=bins);
plt.xticks(np.arange(-4, 4, 1));

This looks like a pleasingly normal, unimodal distribution around 0, which makes sense given that the metric is constructed with zero being the score of an average OECD student and one being the  standard deviation across equally weighted OECD countries.

In [ ]:
#graph all scores together for convenience
sb.violinplot(data=df[['MathScore','ReadScore','ScienceScore']]);

### Discuss the distribution(s) of your variable(s) of interest. Were there any unusual points? Did you need to perform any transformations?
For `Country`, the distribution of # of students sampled may appear to be logarithmic, but it's important to remember that this is a bar chart and not a histogram. I don't think a transformation would be helpful for that reason. Adding a line to indicate the 5,000 suggested minimum sample size may help. For `Sex`, there were slightly more female (50.18%) than male (49.82%) students. The remaining metrics of `SocioEconIndex`, `MathScore`, `ReadScore`, and `ScienceScore` are constructed scores that are designed to have normal distributions, so there was nothing unusual about them and did not require any transformations.

### Of the features you investigated, were there any unusual distributions? Did you perform any operations on the data to tidy, adjust, or change the form of the data? If so, why did you do this?
The `Country` distribution is a bit strange due to some countries opting to test more students (see notes above), but I think showing it in both bar and pie chart forms helps clarify the discrepancies. I did perform operations on the dataset as a whole to eliminate countries with partial testing, and non-OECD countries as they tend to have more erratic results. The `SocioEconIndex` distribution had a minimum of -5.62, but I opted to start the histogram at -4 to show more detail because the y-axis scale would not even show the points below -4.

## Bivariate Exploration

> In this section, investigate relationships between pairs of variables in your
data. Make sure the variables that you cover here have been introduced in some
fashion in the previous section (univariate exploration).

### Talk about some of the relationships you observed in this part of the investigation. How did the feature(s) of interest vary with other features in the dataset?

> Your answer here!

### Did you observe any interesting relationships between the other features (not the main feature(s) of interest)?

> Your answer here!

## Multivariate Exploration

> Create plots of three or more variables to investigate your data even
further. Make sure that your investigations are justified, and follow from
your work in the previous sections.

### Talk about some of the relationships you observed in this part of the investigation. Were there features that strengthened each other in terms of looking at your feature(s) of interest?

> Your answer here!

### Were there any interesting or surprising interactions between features?

> Your answer here!

> At the end of your report, make sure that you export the notebook as an
html file from the `File > Download as... > HTML` menu. Make sure you keep
track of where the exported file goes, so you can put it in the same folder
as this notebook for project submission. Also, make sure you remove all of
the quote-formatted guide notes like this one before you finish your report!